<a href="https://colab.research.google.com/github/Amplil/nlp-test/blob/master/transformer_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Transformerを用いたニューラル機械翻訳をやってみよう](https://qiita.com/migihito_ccc/items/d1c559888e240e8bf535)を参考にしたプログラム

In [2]:
ls

drive/  sample_data/


In [3]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")

# コーパスのダウンロード

In [5]:
import os
import urllib.request
import zipfile
import tarfile

In [4]:
!wget http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz

--2021-05-24 10:40:09--  http://www.phontron.com/kftt/download/kftt-data-1.0.tar.gz
Resolving www.phontron.com (www.phontron.com)... 208.113.196.149
Connecting to www.phontron.com (www.phontron.com)|208.113.196.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99246893 (95M) [application/gzip]
Saving to: ‘kftt-data-1.0.tar.gz’

kftt-data-1.0.tar.g 100%[===================>]  94.65M  14.4MB/s    in 8.1s    

2021-05-24 10:40:18 (11.7 MB/s) - ‘kftt-data-1.0.tar.gz’ saved [99246893/99246893]



In [8]:
# mask.tar.gzの解凍
with tarfile.open('kftt-data-1.0.tar.gz', 'r:*') as tar:
    tar.extractall()

In [12]:
!cp -r kftt-data-1.0 "/content/drive/MyDrive/Colab Notebooks/data"

In [14]:
# !pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" sentencepiece

     |████████████████████████████████| 1.2MB 3.0MB/s 


In [19]:
f = open('./kftt-data-1.0/data/orig/kyoto-dev.en', 'r')
f.read()

'Rinzai Zen Buddhism is one of the Chinese five Houses/seven Schools of Zen (Rinzai, Igyou, Soto, Unmon, Hogan) and Gigen RINZAI (? - 867) of Tang was its founder.\nHe was known as "RINZAI of Katu (meaning to help wake up the sleeping mind) " and "General RINZAI", and rose to the top of Chinese Zen with his distinctive style.\nWith its Zen Talks that try to awaken self awareness through study of Koan, it differs from the Silent Zen of the Soto sect, which just does meditation.\nRinzai School in China\nAs the name implies, Rinzai School started with its founder Gigen RINZAI at the end of the Tang dynasty, after the anti-Buddhist movement at the Kaishou era.\nRINZAI was the disciple of Kiun OBAKU; the school grew as a religious movement centered in the province of Hebei, with the support of Changshi WANG of the Hanchin government, but in the confusion that arose around the end of the Tang dynasty, Hebei was at the center of the tumultuous five dynasties and it became difficult to sustain

In [18]:
f = open('./kftt-data-1.0/data/tok/kyoto-dev.en', 'r')
f.read()

'Rinzai Zen Buddhism is one of the Chinese five Houses / seven Schools of Zen ( Rinzai , Igyou , Soto , Unmon , Hogan ) and Gigen RINZAI ( ? - 867 ) of Tang was its founder .\nHe was known as " RINZAI of Katu ( meaning to help wake up the sleeping mind ) " and " General RINZAI " , and rose to the top of Chinese Zen with his distinctive style .\nWith its Zen Talks that try to awaken self awareness through study of Koan , it differs from the Silent Zen of the Soto sect , which just does meditation .\nRinzai School in China\nAs the name implies , Rinzai School started with its founder Gigen RINZAI at the end of the Tang dynasty , after the anti-Buddhist movement at the Kaishou era .\nRINZAI was the disciple of Kiun OBAKU ; the school grew as a religious movement centered in the province of Hebei , with the support of Changshi WANG of the Hanchin government , but in the confusion that arose around the end of the Tang dynasty , Hebei was at the center of the tumultuous five dynasties and it

In [20]:
f = open('./kftt-data-1.0/data/orig/kyoto-dev.ja', 'r')
f.read()

'臨済宗（臨濟宗、りんざいしゅう）は、中国禅五家七宗（ごけしちしゅう）（臨済、潙仰宗、曹洞宗、雲門宗、法眼宗）のひとつで、唐の臨済義玄（？-867年）を宗祖とする。\n彼は『喝の臨済』『臨済将軍』の異名で知られ、豪放な家風を特徴として中国禅興隆の頂点を極めた。\n公案に参究することにより見性しようとする看話禅（かんなぜん）で、ただ座禅する曹洞宗の黙照禅とはこの点が異なる。\n中国における臨済宗\n臨済宗は、その名の通り、会昌の廃仏後、唐末の宗祖臨済義玄に始まる。\n臨済は黄檗希運の弟子であり、河北省の地を拠点とし、新興の藩鎮勢力であった成徳府藩鎮の王常侍を支持基盤として宗勢を伸張したが、唐末五代の混乱した時期には、河北は5王朝を中心に混乱した地域であったため、宗勢が振るわなくなる。\nこの時期の中心人物は、風穴延昭である。\n臨済宗が再び活気に満ち溢れるようになるのは、北宋代であり、石霜楚円の門下より、ともに江西省を出自とする、黄龍慧南と楊岐方会という、臨済宗の主流となる2派（黄龍派・楊岐派）を生む傑僧が出て、中国全土を席巻することとなった。\n南宋代になると、楊岐派に属する圜悟克勤の弟子の大慧宗杲が、浙江省を拠点として大慧派を形成し、臨済宗の中の主流派となった。\n日本における臨済宗\n宗門では、ゴータマ・シッダッタの教え（悟り）を直接に受け継いだ十大弟子（迦葉）から28代目のボーディダルマ（菩提達磨）を得てインドから中国に伝えられた、ということになっている。\nその後、禅宗の最高峰を極めた臨済宗は、南宋時代の中国に渡り学んだ栄西らによって、鎌倉時代に日本に伝えられている。\n日本の臨済宗は、日本の禅の宗派のひとつである。\n師から弟子への悟りの伝達（法嗣、はっす）を重んじる。\n釈迦を本師釈迦如来大和尚と、ボーディダルマを初祖菩提達磨大師、臨済を宗祖臨済大師と呼ぶ。\n同じ禅宗の曹洞宗が地方豪族や一般民衆に広まったのに対し、臨済宗は時の武家政権に支持され、政治・文化に重んじられた。\nその後時代を下り、江戸時代に白隠禅師によって臨済宗が再建されたため、現在の臨済禅は白隠禅ともいわれている。\n伝統\n法嗣という師匠から弟子へと悟りの伝達が続き現在に至る。\n師匠と弟子の重要なやりとりは、室内の秘密と呼ばれ師匠の部屋の中から持ち出されて公開されることはない。\n師

In [21]:
f = open('./kftt-data-1.0/data/tok/kyoto-dev.ja', 'r')
f.read()

'臨済 宗 （ 臨濟 宗 、 りんざい しゅう ） は 、 中国 禅 五 家 七 宗 （ ご け しち しゅう ） （ 臨済 、 潙仰 宗 、 曹洞 宗 、 雲門 宗 、 法眼 宗 ） の ひと つ で 、 唐 の 臨済 義 玄 （ ？ -867 年 ） を 宗祖 と する 。\n彼 は 『 喝 の 臨済 』 『 臨済 将軍 』 の 異名 で 知 ら れ 、 豪放 な 家風 を 特徴 と し て 中国 禅 興隆 の 頂点 を 極め た 。\n公案 に 参究 する こと に よ り 見性 し よ う と する 看話 禅 （ かんな ぜん ） で 、 ただ 座禅 する 曹洞 宗 の 黙照 禅 と は この 点 が 異な る 。\n中国 に おけ る 臨済 宗\n臨済 宗 は 、 その 名 の 通り 、 会昌 の 廃仏 後 、 唐 末 の 宗祖 臨済 義 玄 に 始ま る 。\n臨済 は 黄檗 希 運 の 弟子 で あ り 、 河北 省 の 地 を 拠点 と し 、 新興 の 藩鎮 勢力 で あ っ た 成徳 府 藩鎮 の 王 常侍 を 支持 基盤 と し て 宗勢 を 伸張 し た が 、 唐 末 五代 の 混乱 し た 時期 に は 、 河北 は 5 王朝 を 中心 に 混乱 し た 地域 で あ っ た ため 、 宗勢 が 振る わ な く な る 。\nこの 時期 の 中心 人物 は 、 風穴 延昭 で あ る 。\n臨済 宗 が 再び 活気 に 満ち溢れ る よう に な る の は 、 北宋 代 で あ り 、 石霜 楚 円 の 門下 より 、 とも に 江西 省 を 出自 と する 、 黄龍 慧 南 と 楊岐 方 会 と い う 、 臨済 宗 の 主流 と な る 2 派 （ 黄龍 派 ・ 楊岐 派 ） を 生 む 傑僧 が 出 て 、 中国 全土 を 席巻 する こと と な っ た 。\n南宋 代 に な る と 、 楊岐 派 に 属 する 圜悟 克 勤 の 弟子 の 大慧 宗杲 が 、 浙江 省 を 拠点 と し て 大慧 派 を 形成 し 、 臨済 宗 の 中 の 主流 派 と な っ た 。\n日本 に おけ る 臨済 宗\n宗門 で は 、 ゴータマ ・ シッダッタ の 教え （ 悟り ） を 直接 に 受け継 い だ 十 大 弟子 （ 迦葉 ）

# OpenNMT

In [ ]:
# トークナイズ
# tokに入っているファイルはすでにトークナイズされている
"""
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
spm.SentencePieceTrainer.Train("--input=tmp.txt --model_prefix=spm_trained_model --vocab_size=16000")
"""

In [22]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 17374, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 17374 (delta 182), reused 290 (delta 162), pack-reused 17044
Receiving objects: 100% (17374/17374), 273.66 MiB | 25.11 MiB/s, done.
Resolving deltas: 100% (12482/12482), done.


In [23]:
cd OpenNMT-py

/content/OpenNMT-py


In [24]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating OpenNMT_py.egg-info
writing OpenNMT_py.egg-info/PKG-INFO
writing dependency_links to OpenNMT_py.egg-info/dependency_links.txt
writing entry points to OpenNMT_py.egg-info/entry_points.txt
writing requirements to OpenNMT_py.egg-info/requires.txt
writing top-level names to OpenNMT_py.egg-info/top_level.txt
writing manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
adding license file 'LICENSE.md' (matched pattern 'LICEN[CS]E*')
writing manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/onmt
copying onmt/trainer.py -> build/lib/onmt
copying onmt/opts.py -> build/lib/onmt
copying onmt/train_single.py -> build/lib/onmt
copying onmt/model_builder.py -> build/lib/onmt
copying onmt/__init__.py -> build/lib/onmt
copying onmt/constants.py -> build/lib/onmt
creating build/lib/onmt/bin
copying onmt/bin

In [26]:
!export CUDA_VISIBLE_DEVICES=0 && \
python train.py -data preprocessed_dataset -save_model save_model_name \
-layers 6 -rnn_size 512 -word_vec_size 512 \
-transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding \
-train_steps 200000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens \
-normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam \
-warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot \
-label_smoothing 0.1 -valid_steps 10000 -save_checkpoint_steps 40000 -world_size 1 -gpu_ranks 0 

Traceback (most recent call last):
  File "train.py", line 2, in <module>
    from onmt.bin.train import main
  File "/content/OpenNMT-py/onmt/__init__.py", line 2, in <module>
    import onmt.inputters
  File "/content/OpenNMT-py/onmt/inputters/__init__.py", line 6, in <module>
    from onmt.inputters.inputter import get_fields, build_vocab, filter_example
  File "/content/OpenNMT-py/onmt/inputters/inputter.py", line 9, in <module>
    from torchtext.data import Field, RawField, LabelField
ImportError: cannot import name 'Field' from 'torchtext.data' (/usr/local/lib/python3.7/dist-packages/torchtext/data/__init__.py)
